<a href="https://colab.research.google.com/github/harmanbirstudy/LLMsPractice/blob/main/gemini2_0_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install google-genai

In [6]:
from google import genai


In [10]:
from google.colab import userdata

client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))



In [ ]:
response = client.models.generate_content(model='gemini-2.0-flash-exp', contents='How does AI work?')
print(response.text)